image classification

In [1]:
import os
import numpy as np
import pandas as pd
import pydicom as pdcm
import cv2
from sklearn.model_selection import train_test_split

def np_CountUpContinuingOnes(b_arr):
    # indice continuing zeros from left side.
    # ex: [0,1,1,0,1,0,0,1,1,1,0] -> [0,0,0,3,3,5,6,6,6,6,10]
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)

    # from right side.
    # ex: [0,1,1,0,1,0,0,1,1,1,0] -> [0,3,3,3,5,5,6,10,10,10,10]
    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]

    return right - left - 1


def ExtractBreast(img):
    img_copy = img.copy()
    img = np.where(img <= 20, 0, img)  # To detect backgrounds easily
    height, _ = img.shape

    # whether each col is non-constant or not
    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    # longest should be the breast
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]

    # whether each row is non-constant or not
    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    # longest should be the breast
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]

    return img_copy[row_ind][:, col_ind]

# 读取XLSX文件
xlsx_path = '/Volumes/图图/DMID-kaggle/archive/Metadata.xlsx'
df = pd.read_excel(xlsx_path)
# df['pathology'] = df['pathology'].fillna('N')

# 排序并按8:2比例分成训练集和测试集
unique_ids = df['ID'].drop_duplicates()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.2, random_state=42)

# 根据去重后的ID划分数据集
train_df = df[df['ID'].isin(train_ids)]
test_df = df[df['ID'].isin(test_ids)]

# 定义输入和输出路径
DCM_PATH = "/Volumes/图图/DMID-kaggle/archive/DICOM Images/DICOM Images"
OUTPUT_BASE_PATH = "/Volumes/图图/DMID-kaggle/archive/image-classification"

def process_and_save(df, set_type):
    for index, row in df.iterrows():
        file_id = row['ID'].strip()
        view_dir=str(row['view']).replace(' ','')
        if view_dir.startswith('MLO'):
            view='MLO'
        elif view_dir.startswith('CC'):
            view='CC'
        if view_dir.endswith('LT'):
            left_or_right='Left'
        elif view_dir.endswith('RT'):
            left_or_right='Right'
        background_tissue = str(row['background tissue']).replace(' ','')
        # abnormality = row['abnormality']
        # pathology = row['pathology']
        meta_data = {
            'view':view,
            'left_or_right':left_or_right,
            'background_tissue': background_tissue
            # 'abnormality': abnormality,
            # 'pathology': pathology
        }
        
        dcm_file = os.path.join(DCM_PATH, f"{file_id}.dcm")
        if os.path.exists(dcm_file):
            dcm = pdcm.dcmread(dcm_file)
            img = dcm.pixel_array
            if len(img.shape) == 3 and img.shape[2] == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            img = ExtractBreast(img)
            img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            # 创建输出目录
            output_dir = os.path.join(OUTPUT_BASE_PATH, set_type, file_id)
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            
            # 保存预处理后的图像为JPG文件
            jpg_path = os.path.join(output_dir, 'img.jpg')
            cv2.imwrite(jpg_path, img)
            
            # 保存元数据为NPY文件
            npy_path = os.path.join(output_dir, 'info_dict.npy')
            np.save(npy_path, meta_data)

            print(f"Processed {file_id} for {set_type} set")
        else:
            print(f"DICOM file for {file_id} not found.")

# 处理并保存训练集和测试集
process_and_save(train_df, 'Train')
process_and_save(test_df, 'Test')


Processed IMG002 for Train set
Processed IMG004 for Train set
Processed IMG005 for Train set
Processed IMG006 for Train set
Processed IMG007 for Train set
Processed IMG008 for Train set
Processed IMG009 for Train set
Processed IMG011 for Train set
Processed IMG011 for Train set
Processed IMG013 for Train set
Processed IMG014 for Train set
Processed IMG015 for Train set
Processed IMG016 for Train set
Processed IMG017 for Train set
Processed IMG018 for Train set
Processed IMG020 for Train set
Processed IMG021 for Train set
Processed IMG022 for Train set
Processed IMG024 for Train set
Processed IMG024 for Train set
Processed IMG025 for Train set
Processed IMG025 for Train set
Processed IMG026 for Train set
Processed IMG027 for Train set
Processed IMG028 for Train set
Processed IMG028 for Train set
Processed IMG029 for Train set
Processed IMG029 for Train set
Processed IMG029 for Train set
Processed IMG030 for Train set
Processed IMG030 for Train set
Processed IMG032 for Train set
Processe

cropped classification pathology

In [15]:
import os
import numpy as np
import pandas as pd
import pydicom as pdcm
import cv2
from sklearn.model_selection import train_test_split

def crop_and_save(img, x, y, radius, output_path, file_id):
    flag = True
    try:
        x = int(x)
        y = int(y)
        radius = int(radius)
        x1 = max(x - radius, 0)
        y1 = max(y - radius, 0)
        x2 = min(x + radius, img.shape[1])
        y2 = min(y + radius, img.shape[0])

        # 裁剪图像
        cropped_img = img[y1:y2, x1:x2]
        cropped_img = cv2.normalize(cropped_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        if not os.path.exists('/'.join(output_path.split('/')[:-1])):
            os.makedirs('/'.join(output_path.split('/')[:-1]))
        
        cv2.imwrite(output_path, cropped_img)
    except:
        flag = False
    return flag

# 读取XLSX文件
xlsx_path = '/Volumes/图图/DMID-kaggle/archive/Metadata.xlsx'
df = pd.read_excel(xlsx_path)
df = df[df['abnormality'] != 'NORM']

# 排序并按8:2比例分成训练集和测试集
df_sorted = df.sort_values(by='ID')
train_df, test_df = train_test_split(df_sorted, test_size=0.2, random_state=42)

# 定义输入和输出路径
DCM_PATH = "/Volumes/图图/DMID-kaggle/archive/DICOM Images/DICOM Images"
OUTPUT_BASE_PATH = "/Volumes/图图/DMID-kaggle/archive/cropped-classification-pathology"

def process_and_save(df, set_type):
    id_counter = {}
    for index, row in df.iterrows():
        pathology = str(row['pathology']).replace(' ', '')
        
        # 跳过 pathology 为 'N' 的行
        if pathology == 'N':
            continue

        file_id = row['ID'].strip()
        abnormality = str(row['abnormality']).replace(' ','')
        x = row['x']
        y = row['y']
        radius = row['radius']
        meta_data = {
            'pathology': pathology
        }
        
        if file_id != 'IMG125' and file_id != 'IMG068' and file_id != 'IMG156' and file_id != 'IMG436' and file_id != 'IMG055' and file_id != 'IMG070' and file_id != 'IMG435' and file_id != 'IMG454':
            dcm_file = os.path.join(DCM_PATH, f"{file_id}.dcm")
            if os.path.exists(dcm_file):
                dcm = pdcm.dcmread(dcm_file)
                img = dcm.pixel_array
                if len(img.shape) == 3 and img.shape[2] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # 更新ID计数器
                if file_id in id_counter:
                    id_counter[file_id] += 1
                else:
                    id_counter[file_id] = 1
                file_id_with_suffix = f"{file_id}_{id_counter[file_id]}"

                # 创建输出目录
                output_dir = os.path.join(OUTPUT_BASE_PATH, set_type, file_id_with_suffix)
                
                # 裁剪并保存预处理后的图像为JPG文件
                jpg_path = os.path.join(output_dir, 'img.jpg')
                flag = crop_and_save(img, x, y, radius, jpg_path, file_id)

                # 保存元数据为NPY文件
                if flag:
                    npy_path = os.path.join(output_dir, 'info_dict.npy')
                    np.save(npy_path, meta_data)

                    print(f"Processed {file_id} for {set_type} set")
            else:
                print(f"DICOM file for {file_id} not found.")

# 处理并保存训练集和测试集
process_and_save(train_df, 'Train')
process_and_save(test_df, 'Test')


Processed IMG024 for Train set
Processed IMG067 for Train set
Processed IMG206 for Train set
Processed IMG305 for Train set
Processed IMG033 for Train set
Processed IMG369 for Train set
Processed IMG421 for Train set
Processed IMG232 for Train set
Processed IMG355 for Train set
Processed IMG101 for Train set
Processed IMG297 for Train set
Processed IMG279 for Train set
Processed IMG451 for Train set
Processed IMG407 for Train set
Processed IMG205 for Train set
Processed IMG114 for Train set
Processed IMG500 for Train set
Processed IMG011 for Train set
Processed IMG053 for Train set
Processed IMG172 for Train set
Processed IMG465 for Train set
Processed IMG446 for Train set
Processed IMG342 for Train set
Processed IMG024 for Train set
Processed IMG482 for Train set
Processed IMG136 for Train set
Processed IMG407 for Train set
Processed IMG010 for Train set
Processed IMG025 for Train set
Processed IMG304 for Train set
Processed IMG063 for Train set
Processed IMG288 for Train set
Processe

cropped classification abnormality

In [13]:
import os
import numpy as np
import pandas as pd
import pydicom as pdcm
import cv2
from sklearn.model_selection import train_test_split

def crop_and_save(img, x, y, radius, output_path, file_id):
    flag = True
    try:
        x = int(x)
        y = int(y)
        radius = int(radius)
        x1 = max(x - radius, 0)
        y1 = max(y - radius, 0)
        x2 = min(x + radius, img.shape[1])
        y2 = min(y + radius, img.shape[0])

        # 裁剪图像
        cropped_img = img[y1:y2, x1:x2]
        cropped_img = cv2.normalize(cropped_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        if not os.path.exists('/'.join(output_path.split('/')[:-1])):
            os.makedirs('/'.join(output_path.split('/')[:-1]))
        
        cv2.imwrite(output_path, cropped_img)
    except:
        flag = False
    return flag

# 读取XLSX文件
xlsx_path = '/Volumes/图图/DMID-kaggle/archive/Metadata.xlsx'
df = pd.read_excel(xlsx_path)
df = df[df['abnormality'].replace(' ', '') != 'NORM']
# 排序并按8:2比例分成训练集和测试集
df_sorted = df.sort_values(by='ID')
train_df, test_df = train_test_split(df_sorted, test_size=0.2, random_state=42)

# 定义输入和输出路径
DCM_PATH = "/Volumes/图图/DMID-kaggle/archive/DICOM Images/DICOM Images"
OUTPUT_BASE_PATH = "/Volumes/图图/DMID-kaggle/archive/cropped-classification-abnormality"

def process_and_save(df, set_type):
    id_counter = {}
    for index, row in df.iterrows():
        # pathology = str(row['pathology']).replace(' ', '')
        abnormality = str(row['abnormality']).replace(' ', '')

        # 跳过 pathology 为 'N' 的行
        # if pathology == 'N':
        #     continue

        # 跳过 abnormality 中包含加号 '+' 的行
        

        file_id = row['ID'].strip()
        x = row['x']
        y = row['y']
        radius = row['radius']
        meta_data = {
            'abnormality': abnormality
        }
        
        if '+' in abnormality:
            print(file_id, 'jump')
            continue
        
        if file_id != 'IMG125' and file_id != 'IMG068' and file_id != 'IMG156' and file_id != 'IMG436' and file_id != 'IMG055' and file_id != 'IMG070' and file_id != 'IMG435' and file_id != 'IMG454':
            dcm_file = os.path.join(DCM_PATH, f"{file_id}.dcm")
            if os.path.exists(dcm_file):
                dcm = pdcm.dcmread(dcm_file)
                img = dcm.pixel_array
                if len(img.shape) == 3 and img.shape[2] == 3:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                
                # 更新ID计数器
                if file_id in id_counter:
                    id_counter[file_id] += 1
                else:
                    id_counter[file_id] = 1
                file_id_with_suffix = f"{file_id}_{id_counter[file_id]}"

                # 创建输出目录
                output_dir = os.path.join(OUTPUT_BASE_PATH, set_type, file_id_with_suffix)
                
                # 裁剪并保存预处理后的图像为JPG文件
                jpg_path = os.path.join(output_dir, 'img.jpg')
                
                flag = crop_and_save(img, x, y, radius, jpg_path, file_id)
                print(file_id_with_suffix, flag)
                # 保存元数据为NPY文件
                if flag:
                    npy_path = os.path.join(output_dir, 'info_dict.npy')
                    np.save(npy_path, meta_data)

                    print(f"Processed {file_id} for {set_type} set")
            else:
                print(f"DICOM file for {file_id} not found.")

# 处理并保存训练集和测试集
process_and_save(train_df, 'Train')
process_and_save(test_df, 'Test')


IMG024_1 True
Processed IMG024 for Train set
IMG067_1 True
Processed IMG067 for Train set
IMG206_1 True
Processed IMG206 for Train set
IMG305 jump
IMG033 jump
IMG085_1 True
Processed IMG085 for Train set
IMG369_1 True
Processed IMG369 for Train set
IMG421_1 True
Processed IMG421 for Train set
IMG232_1 True
Processed IMG232 for Train set
IMG355_1 True
Processed IMG355 for Train set
IMG101_1 True
Processed IMG101 for Train set
IMG297 jump
IMG279_1 True
Processed IMG279 for Train set
IMG451_1 True
Processed IMG451 for Train set
IMG407_1 True
Processed IMG407 for Train set
IMG205_1 True
Processed IMG205 for Train set
IMG114_1 True
Processed IMG114 for Train set
IMG500_1 True
Processed IMG500 for Train set
IMG011_1 True
Processed IMG011 for Train set
IMG053_1 True
Processed IMG053 for Train set
IMG172_1 True
Processed IMG172 for Train set
IMG465_1 True
Processed IMG465 for Train set
IMG446_1 True
Processed IMG446 for Train set
IMG342_1 True
Processed IMG342 for Train set
IMG024 jump
IMG482_

detection

In [3]:
import os
import pandas as pd
import numpy as np
import cv2
import pydicom as pdcm
from tqdm import tqdm
from sklearn.model_selection import train_test_split

def np_CountUpContinuingOnes(b_arr):
    left = np.arange(len(b_arr))
    left[b_arr > 0] = 0
    left = np.maximum.accumulate(left)

    rev_arr = b_arr[::-1]
    right = np.arange(len(rev_arr))
    right[rev_arr > 0] = 0
    right = np.maximum.accumulate(right)
    right = len(rev_arr) - 1 - right[::-1]

    return right - left - 1

def adjust_bounding_box(original_coords, left_crop, top_crop):
    x1, y1, x2, y2 = original_coords

    x1_new = x1 - left_crop
    y1_new = y1 - top_crop
    x2_new = x2 - left_crop
    y2_new = y2 - top_crop

    return x1_new, y1_new, x2_new, y2_new

def ExtractBreast(img, true_bounding_box):
    img_copy = img.copy()
    img = np.where(img <= 40, 0, img)  # To detect backgrounds easily
    height, _ = img.shape

    y_a = height // 2 + int(height * 0.4)
    y_b = height // 2 - int(height * 0.4)
    b_arr = img[y_b:y_a].std(axis=0) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    col_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[:, col_ind]

    _, width = img.shape
    x_a = width // 2 + int(width * 0.4)
    x_b = width // 2 - int(width * 0.4)
    b_arr = img[:, x_b:x_a].std(axis=1) != 0
    continuing_ones = np_CountUpContinuingOnes(b_arr)
    row_ind = np.where(continuing_ones == continuing_ones.max())[0]
    img = img[row_ind, :]

    adjusted_bboxes = []
    for bbox in true_bounding_box:
        adjusted_coords = adjust_bounding_box(bbox, col_ind[0], row_ind[0])
        adjusted_bboxes.append(adjusted_coords)

    return img_copy[row_ind][:, col_ind], adjusted_bboxes

# 加载xlsx文件
xlsx_path = '/Volumes/图图/DMID-kaggle/archive/Metadata.xlsx'
df = pd.read_excel(xlsx_path)

# 初始化字典保存数据
cropped = {}
data_path = '/Volumes/图图/DMID-kaggle/archive/DICOM Images/DICOM Images'

# 遍历DataFrame的每一行
for index, row in tqdm(df.iterrows()):
    img_id = row['ID'].strip()
    x_value = row['x']
    abnormality = row['abnormality'].strip()
    
    # 如果x不为空，且abnormality中不包含'+'
    if pd.notna(x_value) and '+' not in abnormality:
        dcm_path = os.path.join(data_path, f'{img_id}.dcm')
        dcm = pdcm.dcmread(dcm_path)
        img = dcm.pixel_array
        if len(img.shape) == 3 and img.shape[2] == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        
        try:
            x = int(row['x'])
            y = int(row['y'])
            radius = int(row['radius'])
            x1 = max(x - radius, 0)
            y1 = max(y - radius, 0)
            x2 = min(x + radius, img.shape[1])
            y2 = min(y + radius, img.shape[0])
            cropped_image = img[y1:y2, x1:x2]
            if cropped_image.shape[0] == 0 or cropped_image.shape[1] == 0:
                continue
            true_bounding_box = [(x1, y1, x2, y2)]
        except:
            continue
        
        preprocessed_img, adjusted_bboxes = ExtractBreast(img, true_bounding_box)
        
        if img_id not in cropped:
            cropped[img_id] = []
        
        cropped[img_id].append((preprocessed_img, adjusted_bboxes, abnormality))
                    
# 分割训练集和测试集
train_ids, test_ids = train_test_split(list(cropped.keys()), test_size=0.2, random_state=42)

# 保存数据的函数
def save_data(img_id, boxes, output_base_path):
    folder_type = "Train" if img_id in train_ids else "Test"
    output_dir = os.path.join(output_base_path, folder_type, img_id)
    os.makedirs(output_dir, exist_ok=True)
    
    for preprocessed_img, adjusted_bboxes, abnormality in boxes:
        jpg_path = os.path.join(output_dir, 'img.jpg')
        cv2.imwrite(jpg_path, cv2.normalize(preprocessed_img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8))
        
        bbox_list = {
            "boxes": adjusted_bboxes,
            "labels": [abnormality] * len(adjusted_bboxes)
        }
        
        bbox_output_path = os.path.join(output_dir, 'bboxes.npy')
        np.save(bbox_output_path, bbox_list)
    
    print(f'Saved {img_id} data to {output_dir}')

# 保存训练集和测试集
target_dir = "/Volumes/图图/DMID-kaggle/archive/multi-obj-det"
for img_id, boxes in cropped.items():
    save_data(img_id, boxes, target_dir)


647it [08:35,  1.25it/s]


Saved IMG001 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG001
Saved IMG008 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG008
Saved IMG010 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG010
Saved IMG011 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG011
Saved IMG012 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG012
Saved IMG014 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG014
Saved IMG015 data to /Volumes/图图/DMID-kaggle/archive/DMID/Test/IMG015
Saved IMG018 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG018
Saved IMG019 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG019
Saved IMG020 data to /Volumes/图图/DMID-kaggle/archive/DMID/Test/IMG020
Saved IMG021 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG021
Saved IMG022 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG022
Saved IMG023 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG023
Saved IMG024 data to /Volumes/图图/DMID-kaggle/archive/DMID/Train/IMG024
Saved IM

In [10]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage import io
from PIL import Image

# 读取Excel文件
excel_path = '/Volumes/图图/DMID-kaggle/archive/Metadata.xlsx'
df = pd.read_excel(excel_path)

# 定义路径
base_image_path = '/Volumes/图图/DMID-kaggle/archive/Pixel-level-annotation/Pixel-level-annotation'
output_path = '/Volumes/图图/DMID-kaggle/archive/sementic-mask'

# 创建输出目录
if not os.path.exists(output_path):
    os.makedirs(output_path)

# 初始化类别字典
classes = {'background': 0}

def determine_class(abnormality):
    """
    将abnormality映射到类别索引。新的类别将自动分配一个新的索引。

    参数:
    abnormality: 异常类型
    
    返回:
    class_id: 类别索引
    """
    if abnormality not in classes:
        classes[abnormality] = len(classes)  # 为新类别分配新的索引
    print(f"Abnormality: {abnormality}, Class ID: {classes[abnormality]}")
    return classes[abnormality]
not_detect=[]
# 遍历所有ID
for index, row in df.iterrows():
    img_id = row['ID'].strip()  # 假设ID列名为'ID'
    x_value = row['x']
    abnormality = row['abnormality'].strip()
    
    # 如果abnormality中包含+号，跳过此行
    if '+' in str(abnormality):
        continue
    
    if pd.notna(x_value):  # 检查x列是否为空
        # 加载图像
        img_path = os.path.join(base_image_path, f'{img_id}.tif')
        
        try:
            image = Image.open(img_path)
        except:
            continue
        image = np.array(image)

        # 创建空白掩码
        mask = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)

        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # 定义红色和绿色的HSV范围
        lower_red1 = np.array([0, 15, 15])
        upper_red1 = np.array([200, 255, 255])
        lower_red2 = np.array([160, 30, 30])
        upper_red2 = np.array([180, 255, 255])
        lower_green = np.array([30, 30, 30])
        upper_green = np.array([90, 255, 255])

        # 过滤红色和绿色
        mask_red1 = cv2.inRange(hsv_image, lower_red1, upper_red1)
        mask_red2 = cv2.inRange(hsv_image, lower_red2, upper_red2)
        mask_red = mask_red1 | mask_red2  # Combine red masks
        mask_green = cv2.inRange(hsv_image, lower_green, upper_green)

        # 合并红色和绿色的掩码
        combined_mask = mask_red | mask_green

        # 使用形态学操作去噪
        kernel = np.ones((5, 5), np.uint8)
        cleaned_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)

        # 查找轮廓
        contours, _ = cv2.findContours(cleaned_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # 计算x, y中心点
        center_x = int(row['x'])
        center_y = int(row['y'])

        # 检查哪个边缘包围了中心点，并生成mask
        found_contour = False
        for contour in contours:
            test_result = cv2.pointPolygonTest(contour, (center_x, center_y), False)
            print(f"Image {img_id}, Center Point ({center_x}, {center_y}), Test Result: {test_result}")
            if test_result >= 0:
                abnormality_class = determine_class(abnormality)
                # 使用cv2.fillPoly将边缘内部的所有像素设置为类别值
                cv2.fillPoly(mask, [contour], abnormality_class)
                found_contour = True
                break  # 一旦找到包围中心点的边缘，就跳出循环
        
        if not found_contour:
            not_detect.append(img_id)
            print(f"No contour found for image {img_id} with center point ({center_x}, {center_y})")

        # 保存生成的mask
        output_mask_path = os.path.join(output_path, f'{img_id}_mask.png')
        io.imsave(output_mask_path, mask)

# 打印生成的类别字典
print('未检测到:',not_detect)
print("生成的类别字典:", classes)


Image IMG001, Center Point (1461, 3102), Test Result: -1.0
Image IMG001, Center Point (1461, 3102), Test Result: 1.0
Abnormality: CIRC, Class ID: 1


/var/folders/ft/tdx4j_ks1s33pn4xyp1nq6km0000gn/T/ipykernel_93420/3843766324.py:109: UserWarning: /Volumes/图图/DMID-kaggle/archive/sementic-mask/IMG001_mask.png is a low contrast image
  io.imsave(output_mask_path, mask)


Image IMG008, Center Point (3211, 3904), Test Result: 1.0
Abnormality: CIRC, Class ID: 1


/var/folders/ft/tdx4j_ks1s33pn4xyp1nq6km0000gn/T/ipykernel_93420/3843766324.py:109: UserWarning: /Volumes/图图/DMID-kaggle/archive/sementic-mask/IMG008_mask.png is a low contrast image
  io.imsave(output_mask_path, mask)


Image IMG010, Center Point (3747, 4647), Test Result: 1.0
Abnormality: CIRC, Class ID: 1


...

Image IMG498, Center Point (3548, 2932), Test Result: -1.0
Image IMG498, Center Point (3548, 2932), Test Result: 1.0
Abnormality: CIRC, Class ID: 1
Image IMG499, Center Point (3333, 2874), Test Result: 1.0
Abnormality: CIRC, Class ID: 1


/var/folders/ft/tdx4j_ks1s33pn4xyp1nq6km0000gn/T/ipykernel_93420/3843766324.py:109: UserWarning: /Volumes/图图/DMID-kaggle/archive/sementic-mask/IMG499_mask.png is a low contrast image
  io.imsave(output_mask_path, mask)


Image IMG500, Center Point (3279, 3913), Test Result: 1.0
Abnormality: CIRC, Class ID: 1


/var/folders/ft/tdx4j_ks1s33pn4xyp1nq6km0000gn/T/ipykernel_93420/3843766324.py:109: UserWarning: /Volumes/图图/DMID-kaggle/archive/sementic-mask/IMG500_mask.png is a low contrast image
  io.imsave(output_mask_path, mask)


未检测到: ['IMG059', 'IMG068', 'IMG070', 'IMG103', 'IMG125', 'IMG125', 'IMG125', 'IMG156', 'IMG356', 'IMG445']
生成的类别字典: {'background': 0, 'CIRC': 1, 'SPIC': 2, 'CALC': 3, 'MISC': 4}
